以下是自动学习框架autogluon的实践。

In [ ]:
%matplotlib inline
from autogluon.vision import ImagePredictor, ImageDataset

In [7]:
train_data, _, test_data = ImageDataset.from_folders('data/cifar', train='train', test='test')
print('train #', len(train_data), 'test #', len(test_data))
train_data.head()

train # 50000 test # 10000


,image,label
0,/home/fangga/dachuang/AI/data/cifar/train/airp...,0
1,/home/fangga/dachuang/AI/data/cifar/train/airp...,0
2,/home/fangga/dachuang/AI/data/cifar/train/airp...,0
3,/home/fangga/dachuang/AI/data/cifar/train/airp...,0
4,/home/fangga/dachuang/AI/data/cifar/train/airp...,0


In [ ]:
predictor = ImagePredictor().fit(train_data=train_data)
predictions = predictor.predict(test_data)

`time_limit=auto` set to `time_limit=7200`.
Reset labels to [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Randomly split train_data into train[45000]/validation[5000] splits.
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != resnet50
root.train.early_stop_baseline 0.0 != -inf
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
root.train.early_stop_patience -1 != 10
root.train.epochs    200 != 15
root.misc.seed       42 != 435
root.misc.num_workers 4 != 16
}
Saved config to /hy-tmp/c148b04f/.trial_0/config.yaml
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth
Model resnet50 created, param count:                                         23528522
AMP not enabled. Training in float32.
Disable EMA as it 

In [ ]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Top-1 train acc: 0.839, val acc: 0.966


In [ ]:
test_acc = predictor.evaluate(test_data)
print('Top-1 test acc: %.3f' % test_acc['top1'])

[Epoch 11] validation: top1=0.961800 top5=0.999400


Top-1 test acc: 0.962


In [ ]:
filename = 'predictor.ag'
predictor.save(filename)

以下是sota算法vision transformers的实践。

In [2]:
!pip intsall vit_pytorch
import torch.nn as nn
import torch
from vit_pytorch.cait import CaiT
# baseline CaiT models pretrained on ImageNet
v = CaiT(
    image_size = 32,
    patch_size = 32,
    num_classes = 10,
    dim = 1024,
    depth = 12,             # depth of transformer for patch to patch attention only
    cls_depth = 2,          # depth of cross attention of CLS tokens to patch
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1,
    layer_dropout = 0.05    # randomly dropout 5% of the layers
)

class Patch_embeded(nn.Module):
    def __init__(self, image_size=224, patch_size=16, embed_dim=768, in_channel=3):
        super().__init__()
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.in_channel  = in_channel
        self.image_size = image_size
        self.num_patches = (self.image_size // self.patch_size) ** 2
        self.proj = nn.Conv2d(self.in_channel, self.embed_dim, kernel_size=self.patch_size, stride=self.patch_size)

        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches+1, self.embed_dim))
    
    def forward(self, x):
        # x: [B, C, H, W]

        x = self.proj(x)
        x = x.flatten(2) # [B, embed_dim, num_patches]
        x = x.transpose(1, 2) # [B, num_patches, embed_dim]

        cls_token = self.cls_token.expand(x.shape[0], -1 ,-1)
        x = torch.cat((x,cls_token), dim=1)  #[B, num_patches+1, embed_dim]
        pos_embed = self.pos_embed

        out = x + pos_embed #[B, num_patches+1, embed_dim]
        return out

class attention(nn.Module):
    def __init__(self, dim, num_heads = 8, qkv_bias = False):
        super().__init__()

        self.num_heads = num_heads
        prehead_dim = dim // self.num_heads
        self.scale = prehead_dim ** -0.5

        self.qkv = nn.Linear(dim, dim*3, bias=qkv_bias)
        self.proj = nn.Linear(dim, dim)

    def forward(self, x):
        # x: [B, num_patches+1, embed_dim]
        B, num_patches, total_dim = x.shape

        qkv = self.qkv(x) #[B, num_patches+1, 3*embed_dim]

        qkv = qkv.reshape(B, num_patches, 3, self.num_heads, total_dim // self.num_heads) #[B, num_patches+1, 3, num_heads, prehead_dim]

        qkv = qkv.permute(2, 0, 3, 1, 4) #[3, B, num_heads, num_patches+1, prehead_dim]
        q, k, v = qkv[0], qkv[1], qkv[2]  #[B, num_heads, num_patches+1, prehead_dim]

        atten = (q @ k.transpose(-2,-1)) * self.scale  #[B, num_heads, num_patches+1, num_patches+1]
        atten = atten.softmax(dim=-1)
        atten = atten @ v  ## [B, num_heads, num_patches+1, prehead_dim]
        atten = atten.transpose(1,2) #[B, num_patches+1, num_heads, prehead_dim]
        atten = atten.reshape(B, num_patches, total_dim)  #[B, num_patches+1, embed_dim]

        out = self.proj(atten)

        return out

class MLP(nn.Module):
    def __init__(self, in_dim, hidden_dim=None, out_dim=None):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.actlayer = nn.GELU()
        self.fc2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        x = self.fc1(x)  #[B, num_patches+1, hidden_dim]
        x = self.actlayer(x)
        x = self.fc2(x)  #[B, num_patches+1, out_dim]
        x = self.actlayer(x)

        return x

class Encoder_block(nn.Module):
    def __init__(self,
                 dim,
                 num_heads, 
                 mlp_ration=4,
                 qkv_bias = False,

                 ):
        super().__init__()

        self.normlayer = nn.LayerNorm(dim)
        self.atten = attention(dim, num_heads, qkv_bias=qkv_bias)
        self.hidden_dim = int(dim*mlp_ration)
        self.mlp = MLP(in_dim = dim, hidden_dim = self.hidden_dim, out_dim=dim)
    
    def forward(self, x):
        x = x + self.atten(self.normlayer(x))
        x = x + self.mlp(self.normlayer(x))

        return x

class Vision_Model(nn.Module):
    def __init__(self,
                in_channel = 3,
                 dim=768,
                 num_heads = 12,
                 image_size = 224,
                 patch_szie = 16,
                 num_classes = 10,
                 depth = 12,
                 qkv_bias = True
                 ):
        super().__init__()
        self.image_size = image_size
        self.patch_size = patch_szie
        self.patch_embed = Patch_embeded(image_size= self.image_size, patch_size=self.patch_size, embed_dim=dim, in_channel=in_channel)
        self.depth = depth
        self.norm = nn.LayerNorm(dim)
        self.encoder = nn.Sequential(*[
            Encoder_block(dim=dim, num_heads= num_heads, mlp_ration=4, qkv_bias=qkv_bias) for i in range(depth)
        ])

        self.head = nn.Linear(dim, num_classes)

    def forward(self,x):

        x = self.patch_embed(x)
        x = self.encoder(x)
        x = self.norm(x)
        x = self.head(x)
        return x

In [48]:
import torchvision as tv
import numpy as np

testset = tv.datasets.ImageFolder(root='./data/cifar/test',transform=tv.transforms.ToTensor())


correct = 0 # 预测正确的图片数
total = 0 # 总共的图片数


# 由于测试的时候不需要求导，可以暂时关闭autograd，提高速度，节约内存
images = []
labels = []
with torch.no_grad():
    for data in testset:
        image, label = data
        image = image
        label = label
        images.append(image)
        labels.append(label)
        total += 1
    # 把images转成tensor
    images = torch.stack(images)
    outputs = v(images)

for i in range(len(outputs)):
        predicted = np.argmax(outputs[i])
        if predicted == labels[i]:
            correct += 1

print('10000张测试集中的准确率为: %d %%' % (100 * correct / total))

10000张测试集中的准确率为: 99.4%
